In [1]:
%pylab
%load_ext autoreload
%autoreload 2

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from metric import tests
import cv2
from utils import wand_transforms
import wand
from wand.image import Image
from metric.separate_iou import siou
from sklearn.metrics import adjusted_rand_score
import tqdm

In [3]:
# generate random images 100
!mkdir -p expr1_data/compare

In [4]:
for i in range(100):
    n_cells = np.random.randint(25, 40)
    circs = tests.uniform_random_circles(n_cells, 55, 450, 10, 50, 5)
    slices = np.zeros((n_cells, 512, 512), np.uint8)
    image = np.zeros((512, 512), np.uint8)
    for j, (c, r) in enumerate(circs):
        slices[j] = cv2.circle(slices[j], c, r, 1, -1)
        image = cv2.circle(image, c, r, 255, -1)
    cv2.imwrite(f'./expr1_data/image_{i}.png', image)
    np.save(f'./expr1_data/slices_{i}.npy', slices)

## Sanity check

In [5]:
lst = ['niou', 'ari', 'f0_ari', 'f1_ari', 'ff_ari', 'siou_normal', 'siou_prop', 'siou_inv']
results = {k:[] for k in lst}
for i in range(10):
    slc = np.load(f'./expr1_data/slices_{i}.npy')
    img = cv2.imread(f'./expr1_data/image_{i}.png', cv2.IMREAD_ANYDEPTH)
    img = (img > 127).astype(uint8)
    for normalize in ['normal', 'prop', 'inv']:
        results['siou_'+normalize].append(siou(img, slc, normalize=normalize))
    
    orig = slc.sum(0).astype(uint8)
    _, x = cv2.connectedComponents(orig, connectivity=8)
    _, y = cv2.connectedComponents(img, connectivity=8)
    results['ari'].append(adjusted_rand_score(x.reshape(-1), y.reshape(-1)))
    
    idx = orig != 0
    results['f0_ari'].append(adjusted_rand_score(x[idx], y[idx]))
    
    idx = img != 0
    results['f1_ari'].append(adjusted_rand_score(x[idx], y[idx]))
    
    idx = (orig != 0) | (y != 0)
    results['ff_ari'].append(adjusted_rand_score(x[idx], y[idx]))
    
    orig = orig.astype(bool)
    img = img.astype(bool)
    results['niou'].append((orig&img).sum()/(orig|img).sum())
    
for k in lst:
    ar = np.array(results[k])
    mean, std = ar.mean(), ar.std()
    print(f'{k:15s} {mean:.2f} {std:.2f}')
    
for k in lst:
    ar = np.array(results[k])
    mean, std = ar.mean(), ar.std()
    print(f'{mean:.2f} $\pm$ {std:.2f}', end=' & ')

niou            1.00 0.00
ari             1.00 0.00
f0_ari          1.00 0.00
f1_ari          1.00 0.00
ff_ari          1.00 0.00
siou_normal     1.00 0.00
siou_prop       1.00 0.00
siou_inv        1.00 0.00
1.00 $\pm$ 0.00 & 1.00 $\pm$ 0.00 & 1.00 $\pm$ 0.00 & 1.00 $\pm$ 0.00 & 1.00 $\pm$ 0.00 & 1.00 $\pm$ 0.00 & 1.00 $\pm$ 0.00 & 1.00 $\pm$ 0.00 & 

## Perturbed versions

In [6]:
tsfm = wand_transforms.Distort(4, 512, 12)
lst = ['niou', 'ari', 'f0_ari', 'f1_ari', 'ff_ari', 'siou_normal', 'siou_prop', 'siou_inv']
results = {k:[] for k in lst}
for i in tqdm.trange(100):
    img = Image(filename=f'./expr1_data/image_{i}.png')
    slc = np.load(f'./expr1_data/slices_{i}.npy')
    img, *_ = tsfm([img])
    img = np.frombuffer(img.make_blob('gray'), uint8).reshape(512,512)
    img = (img > 127).astype(uint8)
    for normalize in ['normal', 'prop', 'inv']:
        results['siou_'+normalize].append(siou(img, slc, normalize=normalize))
    
    tocompare = np.ones((512, 1026), uint8)
    
    orig = slc.sum(0).astype(uint8)
    tocompare[:, :512] = orig
    tocompare[:, 514:] = img
    
    _, x = cv2.connectedComponents(orig, connectivity=8)
    _, y = cv2.connectedComponents(img, connectivity=8)
    results['ari'].append(adjusted_rand_score(x.reshape(-1), y.reshape(-1)))
    
    idx = orig != 0
    results['f0_ari'].append(adjusted_rand_score(x[idx], y[idx]))
    
    idx = img != 0
    results['f1_ari'].append(adjusted_rand_score(x[idx], y[idx]))
    
    idx = (orig != 0) | (y != 0)
    results['ff_ari'].append(adjusted_rand_score(x[idx], y[idx]))
    
    orig = orig.astype(bool)
    img = img.astype(bool)
    results['niou'].append((orig&img).sum()/(orig|img).sum())
    cv2.imwrite(f'./expr1_data/compare/{i}.png', tocompare*255)

for k in lst:
    ar = np.array(results[k])
    mean, std = ar.mean(), ar.std()
    print(f'{k:15s} {mean:.2f} {std:.2f}')
    
for k in lst:
    ar = np.array(results[k])
    mean, std = ar.mean(), ar.std()
    print(f'{mean:.2f} $\pm$ {std:.2f}', end=' & ')

100%|██████████| 100/100 [00:47<00:00,  2.08it/s]

niou            0.79 0.03
ari             0.82 0.03
f0_ari          0.80 0.05
f1_ari          0.79 0.05
ff_ari          0.66 0.07
siou_normal     0.90 0.04
siou_prop       0.96 0.02
siou_inv        0.81 0.06
0.79 $\pm$ 0.03 & 0.82 $\pm$ 0.03 & 0.80 $\pm$ 0.05 & 0.79 $\pm$ 0.05 & 0.66 $\pm$ 0.07 & 0.90 $\pm$ 0.04 & 0.96 $\pm$ 0.02 & 0.81 $\pm$ 0.06 & 